In [1]:
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require(tidyverse)
require(ggplot2)
require(data.table)
require(egg)
require(foreach)
require(doParallel)
require(ggforce)
# registerDoParallel(cores=8)

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: egg

Loading required package: gridExtra


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



In [29]:
df <- fread("data/blood_culture_records/aerobic_2011.csv")
colnames(df) <- tolower(colnames(df))
colnames(df) <- gsub("[[:punct:]]", "", colnames(df))
colnames(df) <- gsub(" ", "_", colnames(df))

df <- df %>% 
    filter(patient_code != "#VALUE!") %>%
    mutate(patient_id = paste0(patient_code, "_", ac_last_4_digits), .before = 1)
head(df)

patient_id,sn,hosp,recv_date,specimen_no,patient_code,ac_last_4_digits,sex,age,race,location,procedure,source,soucat,organisms,status,collection
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AM_142H,1,SGH,21/2/2011,20110011083RB,AM,142H,M,76,I,S73,DBARD,BLP,BL,,COMP,20/2/2011
AM_142H,2,SGH,28/2/2011,20110012588RB,AM,142H,M,76,I,S73,DBARD,BLP,BL,,COMP,27/2/2011
AM_142H,3,SGH,1/3/2011,20110013038RB,AM,142H,M,76,I,S73,DBARD,BLP,BL,,COMP,1/3/2011
AB_493D,4,SGH,7/6/2011,20110034199RB,AB,493D,M,47,M,S55B,DBARD,BLP,BL,,COMP,6/6/2011
AA_431F,5,SGH,22/10/2011,20110064079RB,AA,431F,F,46,O,S78,DBARD,BLP,BL,,COMP,22/10/2011
AA_885B,6,SGH,5/3/2011,20110013885RB,AA,885B,F,36,O,S78,DBARD,BLP,BL,,COMP,4/3/2011


In [32]:
df %>% 
    group_by(patient_id) %>%
    summarise(n = n()) %>%
    arrange(desc(n))

patient_id,n
<chr>,<int>
A_982F,36
JM_130J,35
CM_551I,31
PH_995H,30
TG_175D,29
TK_845E,29
A_151E,28
SS_355I,28
AC_483G,26


In [21]:
length(unique(df$ac_last_4_digits))
length(unique(df$patient_id))
length(df$patient_id)

[1] 10016

[1] 22747

[1] 1048575

In [47]:
unique(df$hosp)

[1] "SGH"  "NCC"  "HSA"  "SNEC" "IND"  "QC"   "BVH"  "NHC"  "PTE1" "PTE" 
[11] "PAP"

In [ ]:
length(df$patient_id)

In [46]:
length(unique(df$specimen_no))
nrow(df)

[1] 37939

[1] 37939

In [42]:
df %>%
    group_by(organisms) %>%
    summarise(n = n()) %>%
    arrange(desc(n))

organisms,n
<chr>,<int>
,34995
ECO,450
SCN,422
SAU,354
MRSA,203
KLE,200
PAE,108
BAC,77
SCNSCN1,59
